In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/traincsv/train.csv


In [11]:
train=pd.read_csv('/kaggle/input/traincsv/train.csv')

In [12]:
train
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621300 entries, 0 to 621299
Columns: 136 entries, SITE_ID to kurt_temperature_alarms_prev14d
dtypes: float64(118), int64(17), object(1)
memory usage: 644.7+ MB


In [13]:
import pandas as pd
import sklearn
import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

data1 = train
data1['DATE'] = pd.to_datetime(data1['DATE'])

y=data1['aircon_sum_target_next14d']

data1.drop(labels='aircon_sum_target_next14d', axis=1, inplace=True)
data1.drop(labels='SITE_ID', axis=1, inplace=True)

data2=data1 

data1['DATE']=data1['DATE'].map(dt.datetime.toordinal)

In [14]:
# Generate and plot a synthetic imbalanced classification dataset
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where

X=data2

#summarize class distribution

counter = Counter(y)
print(counter)

Counter({0: 617717, 1: 3583})


In [15]:
# fit xgboost on an imbalanced classification dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import xgboost
from xgboost import XGBClassifier
# generate dataset
#X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
#	n_clusters_per_class=2, weights=[0.99], flip_y=0, random_state=7)
# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model

In [16]:
#scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
#print('Mean ROC AUC: %.5f' % mean(scores))

In [17]:
# estimate a value for the scale_pos_weight xgboost hyperparameter
from sklearn.datasets import make_classification
from collections import Counter


estimate = 617717/3583
print('Estimate: %.3f' % estimate)

Estimate: 172.402


In [ ]:
# grid search positive class weights with xgboost for imbalance classification
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
# generate dataset
X=data1[['N_TRANSPORTED_SITES', 'kurt_equipment_alarms_prev14d',
       'kurt_power_alarms_prev14d', 'aircon_sum_wo_prev14d',
       'kurt_ge_alarms_prev14d', 'kurt_temperature_alarms_prev14d',
       'kurt_fire/smoke_alarms_prev14d', 'skew_equipment_alarms_prev14d',
       'aircon_sum_wo_prev7d', 'mean_rain_mm_f_next14d',
       'mean_humidity_f_next14d', 'equipment_sum_alarms_prev14d',
       'mean_temperature_f_next14d', 'temperature_sum_alarms_prev14d',
       'skew_temperature_alarms_prev14d', 'mean_wind_speed_f_next14d',
       'max_rain_mm_f_next14d', 'min_humidity_prev7d', 'mean_humidity_prev7d',
       'max_humidity_f_next14d', 'min_humidity_f_next14d',
       'mean_temperature_f_next7d', 'mean_humidity_f_next7d',
       'max_rain_mm_prev7d', 'min_humidity_f_next7d',
       'temperature_mean_persistance_prev7d', 'mean_pressure_f_next14d',
       'mean_rain_mm_f_next7d', 'mean_wind_speed_prev7d',
       'mean_wind_speed_f_next7d']]
# define model
model = XGBClassifier()
# define grid
weights = [ 1, 10, 25, 50, 75, 99, 1000, estimate]
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))